In [1]:
# mlrun: start-code

In [2]:
from cloudpickle import load
import numpy as np
from typing import List, Tuple
import mlrun
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        X = feats[:, :-1]
        y = feats[:, -1]
        y_pred = self.model.predict(X)

        return {
            "prediction": y_pred.tolist(),
            "metrics" : {
                "accuracy" : accuracy_score(y, y_pred),
                "f1" : f1_score(y, y, average="micro"),
                "precision" : precision_score(y, y_pred, average="micro"),
                "recall" : recall_score(y, y_pred, average="micro"),
            }
        }

In [3]:
# mlrun: end-code

In [4]:
from mlrun import code_to_function, auto_mount

serving_fn = code_to_function(
    name='model-server',
    project="royal-cyber",
    kind='serving',
    image='mlrun/mlrun'
).apply(auto_mount())

serving_fn.spec.default_class = 'ClassifierModel'

serving_fn.add_model('lr', model_path="store://models/royal-cyber/my-training-job-tracking-main_LogisticRegression_model#0:latest@32c8cd1816a14b5eb49d99ab92521b08")
serving_fn.add_model('rf', model_path="store://models/royal-cyber/my-training-job-tracking-main_RandomForestClassifier_model#0:latest@32c8cd1816a14b5eb49d99ab92521b08")

In [5]:
function_address = serving_fn.deploy()

> 2021-08-25 21:04:44,565 [info] Starting remote function deploy
2021-08-25 21:04:44  (info) Deploying function
2021-08-25 21:04:44  (info) Building
2021-08-25 21:04:44  (info) Staging files and preparing base images
2021-08-25 21:04:44  (info) Building processor image
2021-08-25 21:04:46  (info) Build complete
2021-08-25 21:04:54  (info) Function deploy complete
> 2021-08-25 21:04:55,157 [info] function deployed, address=default-tenant.app.us-sales-eks.iguazio-cd0.com:30624


In [6]:
import json
import requests
from sklearn.datasets import load_iris

In [7]:
X, y = load_iris(return_X_y=True, as_frame=True)
X_test = pd.concat([X, y.to_frame()], axis=1).sample(frac=1).head(20)

data = json.dumps({"inputs" : X_test.to_dict(orient="split")["data"]})
data

'{"inputs": [[5.2, 2.7, 3.9, 1.4, 1], [6.5, 3.2, 5.1, 2.0, 2], [6.1, 2.9, 4.7, 1.4, 1], [4.6, 3.2, 1.4, 0.2, 0], [5.5, 3.5, 1.3, 0.2, 0], [5.3, 3.7, 1.5, 0.2, 0], [4.8, 3.4, 1.6, 0.2, 0], [6.5, 3.0, 5.2, 2.0, 2], [5.1, 3.4, 1.5, 0.2, 0], [4.7, 3.2, 1.3, 0.2, 0], [5.9, 3.2, 4.8, 1.8, 1], [6.4, 3.2, 5.3, 2.3, 2], [6.3, 3.3, 4.7, 1.6, 1], [4.8, 3.1, 1.6, 0.2, 0], [5.8, 2.6, 4.0, 1.2, 1], [4.9, 3.6, 1.4, 0.1, 0], [6.3, 2.9, 5.6, 1.8, 2], [6.2, 2.9, 4.3, 1.3, 1], [6.6, 3.0, 4.4, 1.4, 1], [6.7, 3.1, 4.4, 1.4, 1]]}'

In [8]:
for model in ["lr", "rf"]:
    # Inference via serving function
    print(serving_fn.invoke(f'/v2/models/{model}/infer', data))
    
    # Inference via HTTP post request
    print(requests.post(url=f"{function_address}/v2/models/{model}/infer", data=data).json())

{'id': 'bd06550b-e002-4c4a-b7dc-11160e87dc08', 'model_name': 'lr', 'outputs': {'prediction': [1, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 1, 0, 1, 0, 2, 1, 1, 1], 'metrics': {'accuracy': 0.95, 'f1': 1.0, 'precision': 0.95, 'recall': 0.95}}}
{'id': 'a5348347-6b4a-4f37-ab62-ebb800f2d45c', 'model_name': 'lr', 'outputs': {'prediction': [1, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 1, 0, 1, 0, 2, 1, 1, 1], 'metrics': {'accuracy': 0.95, 'f1': 1.0, 'precision': 0.95, 'recall': 0.95}}}
{'id': '70e58c4d-b04d-4a86-95e2-5fea08d308e7', 'model_name': 'rf', 'outputs': {'prediction': [1, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 1, 0, 1, 0, 2, 1, 1, 1], 'metrics': {'accuracy': 0.95, 'f1': 1.0, 'precision': 0.95, 'recall': 0.95}}}
{'id': '27355681-2484-4ab6-84d0-09c8f5c54ab7', 'model_name': 'rf', 'outputs': {'prediction': [1, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 1, 0, 1, 0, 2, 1, 1, 1], 'metrics': {'accuracy': 0.95, 'f1': 1.0, 'precision': 0.95, 'recall': 0.95}}}
